In [7]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database
conn = create_engine(string).connect()

test = pd.read_sql(
    """
    select date, ticker, ret, roeq, mom12m
    from data
    where date>='2010-01' 
    order by date, ticker
    """, 
    conn
)
test = test.dropna()

from pandas_datareader import DataReader as pdr

mkt = pdr(
  "F-F_Research_Data_Factors",
  "famafrench",
  start="2009-12",
)

mkt = mkt[0] / 100
mkt.index = mkt.index.astype(str)
mkt["mkt"] = mkt["Mkt-RF"] + mkt["RF"]
mkt["lagmkt"] = mkt.mkt.shift()

df = df.merge(mkt, left_on="date", right_index=True, how="left")
train = df[df.date<="2017-12"]
test = df[df.date>"2017-12"].copy()

In [8]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import Ridge

model = TransformedTargetRegressor(
    transformer=QuantileTransformer(
      output_distribution="normal",
      random_state=0
    ),
    regressor=Ridge(random_state=0)
)

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
  QuantileTransformer(output_distribution="normal", random_state=0),
  PolynomialFeatures(degree=2),
  QuantileTransformer(output_distribution="normal", random_state=0),
  model
)

In [14]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from scipy.stats import spearmanr

spearman = make_scorer(
    lambda y1, y2 : spearmanr(y1, y2)[0]
)

param_grid = {
    "transformedtargetregressor__regressor__alpha": [0.01, 0.1, 1]
}

cv = GridSearchCV(
    pipe, 
    param_grid=param_grid,
    scoring=spearman,
    
)

Xtrain = train[["roeq", "mom12m", "lagmkt"]]
ytrain = train.ret - train.mkt
_ = cv.fit(Xtrain, ytrain)

In [15]:
cv.score(Xtrain, ytrain)

0.07606199941872366

In [16]:
Xtest = test[["roeq", "mom12m", "lagmkt"]]
ytest = test.ret - test.mkt

cv.score(Xtest, ytest)

0.029899619493245655

In [17]:
cv.best_params_

{'transformedtargetregressor__regressor__alpha': 0.1}

In [24]:
test = df[df.date>"2017-12"].copy()
test["predict"] = cv.predict(Xtest)

num = 500

test["rnk"] = test.groupby("date").predict.rank(method="first", ascending=False)
best = test[test.rnk<=num]
test["rnk"] = test.groupby("date").predict.rank(method="first")
worst = test[test.rnk<=num]

best_rets = best.groupby("date").ret.mean()
worst_rets = worst.groupby("date").ret.mean()
rets = pd.concat((best_rets, worst_rets), axis=1)
rets.columns = ["best", "worst"]

In [25]:
rets.mean()

best     0.011068
worst    0.014230
dtype: float64